# **Proyecto Final**

- **Descripción**: El proyecto final consistirá en aplicar una combinación de técnicas aprendidas a lo largo del curso para resolver un problema real de series temporales. Selecciona un conjunto de datos complejo, como precios de acciones, demanda de energía o ventas de productos, y aplica técnicas como ARIMA, SARIMA, GARCH, LSTM, y modelos de machine learning para construir un sistema de predicción integral. Evalúa el rendimiento de cada modelo y proporciona recomendaciones basadas en el análisis comparativo.

**Explicación paso a paso:**

*Importación de datos:* Usamos la API yfinance para obtener los precios históricos de acciones de una empresa. Cambia el ticker para analizar otras acciones.

*División del dataset:* Dividimos los datos en conjunto de entrenamiento (80%) y de prueba (20%) para evaluar los modelos.

*Modelo ARIMA:* Ajustamos un modelo ARIMA básico y calculamos su error RMSE en los datos de prueba.

*Modelo SARIMA:* Extendemos ARIMA añadiendo componentes estacionales para capturar patrones repetitivos.

*Modelo GARCH:* Modelamos la volatilidad, algo clave en series temporales financieras, como precios de acciones.

*Red Neuronal LSTM:* Entrenamos un modelo LSTM, diseñado para manejar secuencias de datos temporales y complejas.

*Comparación de Modelos:* Comparamos los errores de cada modelo y decidimos cuál es más efectivo según el RMSE.

In [1]:
# Importar las bibliotecas necesarias
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from arch import arch_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM

2024-09-24 12:47:08.977347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-24 12:47:10.370240: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-24 12:47:10.592352: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-24 12:47:11.927840: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-24 12:47:23.918177: W tensorflow/compiler/tf2

In [2]:
# Paso 1: Descargar los datos de precios de acciones desde una API

# Usamos la API de Yahoo Finance para descargar datos históricos de una acción específica, en este caso Apple (AAPL)
# Puedes cambiar el ticker por cualquier otra acción
ticker = 'AAPL'
data = yf.download(ticker, start='2010-01-01', end='2023-01-01')

# Solo necesitamos la columna 'Adj Close' para análisis de series temporales
df = data[['Adj Close']].copy()

# Mostrar las primeras filas de los datos
print(df.head())

[*********************100%***********************]  1 of 1 completed

            Adj Close
Date                 
2010-01-04   6.454504
2010-01-05   6.465664
2010-01-06   6.362821
2010-01-07   6.351057
2010-01-08   6.393281


In [3]:
# Paso 2: División de los datos en conjunto de entrenamiento y prueba
# Vamos a utilizar el 80% de los datos para entrenamiento y el 20% para pruebas
train_size = int(len(df) * 0.8)
train, test = df.iloc[:train_size], df.iloc[train_size:]

In [5]:
# Paso 3: Modelo ARIMA
# ARIMA es una técnica común para el análisis de series temporales. Aquí ajustamos un modelo ARIMA simple.

# Ajustamos un modelo ARIMA al conjunto de entrenamiento
model_arima = ARIMA(train, order=(5,1,0))  # 'order' (p,d,q)
model_arima_fit = model_arima.fit()

# Predecir los valores en el conjunto de prueba
predictions_arima = model_arima_fit.forecast(steps=len(test))

# Evaluamos el error usando RMSE (Root Mean Squared Error)
rmse_arima = np.sqrt(mean_squared_error(test, predictions_arima))
print(f"ARIMA RMSE: {rmse_arima}")

/home/julio/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/julio/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/julio/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


ARIMA RMSE: 63.27262275300672


/home/julio/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/julio/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [6]:
# Paso 4: Modelo SARIMA (ARIMA estacional)
# SARIMA incluye componentes estacionales en ARIMA. Ajustamos el modelo SARIMA para capturar la estacionalidad de los datos.

model_sarima = SARIMAX(train, order=(5,1,0), seasonal_order=(1,1,1,12))
model_sarima_fit = model_sarima.fit()

# Predecir los valores en el conjunto de prueba
predictions_sarima = model_sarima_fit.forecast(steps=len(test))

# Evaluamos el error usando RMSE
rmse_sarima = np.sqrt(mean_squared_error(test, predictions_sarima))
print(f"SARIMA RMSE: {rmse_sarima}")

/home/julio/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/julio/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.07163D+00    |proj g|=  4.04862D-01


 This problem is unconstrained.



At iterate    5    f=  9.79026D-01    |proj g|=  3.97468D-02

At iterate   10    f=  9.53688D-01    |proj g|=  1.35672D-02

At iterate   15    f=  9.52818D-01    |proj g|=  5.24048D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8     19     23      1     0     0   2.109D-06   9.528D-01
  F =  0.95281626734286495     

CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL            
SARIMA RMSE: 43.93930318796436


/home/julio/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/home/julio/.local/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [7]:
# Paso 5: Modelo GARCH
# GARCH es utilizado para modelar la volatilidad en series temporales financieras.

# Ajustamos un modelo GARCH al conjunto de entrenamiento
model_garch = arch_model(train, vol='Garch', p=1, q=1)
model_garch_fit = model_garch.fit(disp="off")

# Predecir la volatilidad en el conjunto de prueba
predictions_garch = model_garch_fit.forecast(horizon=len(test))
garch_variance = predictions_garch.variance.values[-1, :]

In [8]:
# Paso 6: Red Neuronal LSTM
# LSTM (Long Short-Term Memory) es un tipo de red neuronal recurrente (RNN) especialmente buena para series temporales.

# Normalizamos los datos para la LSTM
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(train)

# Creamos las secuencias de entrada/salida para LSTM
def create_sequences(data, seq_length):
    x, y = [], []
    for i in range(len(data)-seq_length):
        x.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(x), np.array(y)

# Definimos la longitud de la secuencia
seq_length = 60
x_train, y_train = create_sequences(scaled_train, seq_length)

# Reshape para el modelo LSTM
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))

# Definimos el modelo LSTM
model_lstm = Sequential()
model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model_lstm.add(LSTM(units=50))
model_lstm.add(Dense(1))

# Compilamos el modelo
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamos el modelo
model_lstm.fit(x_train, y_train, epochs=10, batch_size=32)

# Predecir en el conjunto de prueba
scaled_test = scaler.transform(test)
x_test, y_test = create_sequences(scaled_test, seq_length)
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

predictions_lstm = model_lstm.predict(x_test)
predictions_lstm = scaler.inverse_transform(predictions_lstm)

# Evaluamos el error de LSTM usando RMSE
rmse_lstm = np.sqrt(mean_squared_error(test[seq_length:], predictions_lstm))
print(f"LSTM RMSE: {rmse_lstm}")

/home/julio/.local/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 11s 90ms/step - loss: 0.0409
Epoch 2/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 8s 95ms/step - loss: 4.5758e-04
Epoch 3/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 7s 91ms/step - loss: 4.1549e-04
Epoch 4/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 7s 88ms/step - loss: 4.1399e-04
Epoch 5/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 8s 96ms/step - loss: 3.5945e-04
Epoch 6/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 7s 93ms/step - loss: 3.7289e-04
Epoch 7/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 7s 91ms/step - loss: 3.6402e-04
Epoch 8/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 7s 88ms/step - loss: 3.1187e-04
Epoch 9/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 7s 89ms/step - loss: 2.6396e-04
Epoch 10/10
80/80 ━━━━━━━━━━━━━━━━━━━━ 7s 89ms/step - loss: 2.4199e-04
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
LSTM RMSE: 14.676341317439906


In [9]:
# Paso 7: Comparación de resultados
# Finalmente, comparamos los errores de los diferentes modelos y tomamos una decisión sobre cuál es el mejor.
print(f"Comparación de RMSE:\nARIMA: {rmse_arima}\nSARIMA: {rmse_sarima}\nLSTM: {rmse_lstm}")

Comparación de RMSE:
ARIMA: 63.27262275300672
SARIMA: 43.93930318796436
LSTM: 14.676341317439906


El resultado muestra una comparación de los errores RMSE (Root Mean Squared Error) para tres modelos diferentes: ARIMA, SARIMA y LSTM. El RMSE mide la diferencia promedio entre los valores reales y los valores predichos, por lo que un valor más bajo indica un mejor rendimiento del modelo. A continuación, se interpreta cada resultado:

1. **ARIMA (RMSE: 63.27)**:
   - El modelo ARIMA tiene el error más alto (63.27), lo que sugiere que este modelo no es tan eficaz para predecir los precios de las acciones en este caso.
   - ARIMA es útil cuando la serie temporal no tiene un patrón estacional claro, pero puede tener dificultades con series más complejas que presentan variaciones estacionales o comportamientos no lineales.

2. **SARIMA (RMSE: 43.94)**:
   - El modelo SARIMA reduce el error significativamente en comparación con ARIMA (RMSE de 43.94).
   - SARIMA es una versión extendida de ARIMA que incluye componentes estacionales, lo que le permite manejar patrones repetitivos en los datos. Esto indica que el componente estacional es relevante en los precios de las acciones, mejorando el rendimiento.

3. **LSTM (RMSE: 14.68)**:
   - El modelo LSTM tiene el menor error (14.68), lo que indica que es el mejor predictor de los tres.
   - LSTM, una red neuronal recurrente, es más capaz de capturar patrones complejos y no lineales en las series temporales. Su capacidad para "recordar" dependencias a largo plazo entre datos hace que sea más efectivo para manejar la complejidad de los precios de acciones.

**Conclusión:**
- **LSTM** es el modelo más eficaz en este caso, con un RMSE significativamente menor que ARIMA y SARIMA. Esto sugiere que los datos de precios de acciones tienen características complejas que LSTM es capaz de capturar mejor que los modelos ARIMA y SARIMA.
- **SARIMA** también muestra un buen rendimiento en comparación con ARIMA, lo que indica que hay un componente estacional en los datos que ayuda a mejorar las predicciones.
- **ARIMA**, aunque útil para series temporales más simples, no logra capturar la complejidad de los datos de manera eficiente en este caso.

Por lo tanto, **LSTM** sería la recomendación más adecuada para predecir precios de acciones en este conjunto de datos.